In [1]:
import logging
from utility import add_project_root_to_path

logging.basicConfig(level=logging.ERROR)

add_project_root_to_path()

In [2]:
from historical_data import get_historical_prices_for_two_assets
from datetime import datetime

stable_coin_symbol = "USDT"
A_symbol = "ETH"
B_symbol = "BTC"
interval = "5m"

start_time = datetime(2022, 11, 10, 12, 0, 0)
end_time = datetime(2024, 11, 10, 12, 0, 0)
historical_prices = get_historical_prices_for_two_assets(
    stable_coin_symbol, A_symbol, B_symbol, interval, start_time, end_time
)

historical_prices

A symbol candles count: 210513
B symbol candles count: 210513
Joined candles count: 210512


,time,price_A,price_B
0,2022-11-10 11:00:00,1181.00,16403.74
1,2022-11-10 11:05:00,1182.30,16430.08
2,2022-11-10 11:10:00,1185.77,16454.00
3,2022-11-10 11:15:00,1183.89,16371.47
4,2022-11-10 11:20:00,1178.90,16294.39
...,...,...,...
210507,2024-11-10 10:40:00,3191.52,79430.00
210508,2024-11-10 10:45:00,3195.74,79451.30
210509,2024-11-10 10:50:00,3200.80,79552.03
210510,2024-11-10 10:55:00,3203.40,79544.00


In [3]:
# Ищу промежутки различного поведения цены
import numpy as np

historical_prices["log_price_A"] = np.log(historical_prices["price_A"])
historical_prices["log_price_B"] = np.log(historical_prices["price_B"])
historical_prices["log_return_A"] = historical_prices["log_price_A"].diff()
historical_prices["log_return_B"] = historical_prices["log_price_B"].diff()

# Разделение на недельны интервалы
historical_prices.set_index("time", inplace=True)
weekly_data = historical_prices.resample("7D").agg({
    "log_return_A": ["mean", "std"],
    "log_return_B": ["mean", "std"]
}).reset_index()

weekly_data.columns = ["time", "mean_return_A", "volatility_A", "mean_return_B", "volatility_B"]


def classify_behavior(mean_return, mean_return_mean, volatility, volatility_mean):
    if abs(mean_return) < mean_return_mean and volatility >= volatility_mean:
        return "Высокая волатильность вокруг константы"
    elif abs(mean_return) < mean_return_mean and volatility < volatility_mean:
        return "Штиль"
    elif mean_return > 0 and volatility < volatility_mean:
        return "Рост"
    elif mean_return < 0 and volatility < volatility_mean:
        return "Падение"

In [4]:
weekly_data["behavior_A"] = weekly_data.apply(
    lambda row: classify_behavior(row["mean_return_A"], weekly_data["mean_return_A"].mean(), row["volatility_A"], weekly_data["volatility_A"].mean()), axis=1
)

weekly_data["behavior_B"] = weekly_data.apply(
    lambda row: classify_behavior(row["mean_return_B"], weekly_data["mean_return_B"].mean(), row["volatility_B"], weekly_data["volatility_B"].mean()), axis=1
)

In [5]:
weekly_data

,time,mean_return_A,volatility_A,mean_return_B,volatility_B,behavior_A,behavior_B
0,2022-11-10,0.000016,0.003987,0.000009,0.002983,None,None
1,2022-11-17,-0.000013,0.002031,-0.000002,0.001354,None,Штиль
2,2022-11-24,0.000044,0.001643,0.000016,0.001061,None,Рост
3,2022-12-01,-0.000024,0.001256,-0.000010,0.000825,Падение,Падение
4,2022-12-08,0.000029,0.001386,0.000028,0.000981,Рост,Рост
...,...,...,...,...,...,...,...
100,2024-10-10,0.000048,0.001475,0.000054,0.001295,Рост,Рост
101,2024-10-17,-0.000016,0.001290,-0.000007,0.001033,Падение,Штиль
102,2024-10-24,0.000025,0.001600,0.000040,0.001256,Рост,Рост
103,2024-10-31,0.000012,0.001821,0.000022,0.001559,None,None
